In [1]:
import xml.etree.ElementTree as ET

# read XML data
tree = ET.parse('usa_00006.xml')
root = tree.getroot()

variablesToKeep = ['YEAR', 'SUBSAMP', 'SERIAL', 'HHWT', 'STATEFIP', 'COUNTYFIPS',
    'PERNUM', 'PERWT', 'SEX', 'MARST', 'RACE']
variablesCorrection = {
    'HHTYPE': {
        '0': None,
        '9': None
    },
    'REGION': {
        '97': None,
        '99': None
    },
    'OWNERSHP': {
        '0': None
    },
    'AGE': {
        '000': '0',
        '090': '90',
        '100': '100',
        '112': '112',
        '115': '115'
    },
    'SEX': {
        
    },
    'MARST': {
        
    },
    'RACE': {
        
    },
    'STATEFIP': {
        
    },
    'HISPAN': {
        '9': None
    },
    'BPL': {
        '999': None
    },
    'SCHOOL': {
        '0': None,
        '9': None
    },
    'EDUC': {
        '00': None
    },
    'EDUCD': {
        '000': None,
        '001': None,
        '999': None
    },
    'EMPSTAT': {
        '0': None
    },
    'OCC': {
        '999': None
    },
    'OCC1990': {
        '999': None
    },
    'MIGRATE5': {
        '0': None,
        '9': None
    },
    'MIGRATE1': {
        '0': None,
        '9': None
    },
    'MIGPLAC5': {
        '000': None,
        '999': None
    },
    'MIGPLAC1': {
        '000': None,
        '997': None,
        '999': None
    }
}
otherNullValues = {
    'RENT': [0, 1],
    'VALUEH': [9999999],
    'INCTOT': [9999999]
}


convertDict = {}
variablesForSQL = []

data_desc = root.find("{http://www.icpsr.umich.edu/DDI}dataDscr")
for var in data_desc:
    name = var.get('name')

    # will need to adjust
    if name in variablesCorrection:
        convertDict[name] = {}
        variablesForSQL.append(name)
        for category in var.findall("{http://www.icpsr.umich.edu/DDI}catgry"):

            valtag = category.find("{http://www.icpsr.umich.edu/DDI}catValu")
            labtag = category.find("{http://www.icpsr.umich.edu/DDI}labl")

            if valtag is not None and labtag is not None:
                val = valtag.text
                label = labtag.text

                if name in variablesCorrection and val in variablesCorrection[name]:
                    convertDict[name][val] = variablesCorrection[name][val]
                else:
                    pass
                convertDict[name][val] = label
        for val in variablesCorrection[name]:
            convertDict[name][val] = variablesCorrection[name][val]
    # leave variable as is
    elif name in variablesToKeep or name in otherNullValues:
        variablesForSQL.append(name)
    # not using for now
    else:
        continue

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('usa_00006_filtered.csv')
df = df[variablesForSQL]
df['STATEFIP_orig'] = df['STATEFIP']
df['OCC1990_orig'] = df['OCC1990']
df['EDUCD_orig'] = df['EDUCD']
df['OCC_orig'] = df['OCC']
df['BPL_orig'] = df['BPL']

staysNumeric = ['AGE']


In [3]:
for var in convertDict:
    newDict = {}
    if var in staysNumeric:
        newDict = {int(x):int(y) for x, y in convertDict[var].items()}
    else:
        if df[var].dtype == 'int64':
            newDict = {int(x):y for x, y in convertDict[var].items()}
            #df[var].replace()
        if df[var].dtype == 'float64':
            newDict = {float(x):y for x, y in convertDict[var].items()}
    df[var] = df[var].map(newDict)
    
for var in otherNullValues:
    for nullVal in otherNullValues[var]:
        df.loc[df[var] == nullVal, var] = np.nan

In [4]:
df.to_csv('usa00006_processed.csv')

In [5]:
df[df['STATEFIP'] == 'Colorado']['COUNTYFIPS'].value_counts()

0     50505
69     3065
Name: COUNTYFIPS, dtype: int64

In [6]:
df2 = pd.read_csv('usa_00006_filtered.csv')

In [7]:
df2[df2['STATEFIP'] == 8]['COUNTYFIPS'].value_counts()

0     50505
69     3065
Name: COUNTYFIPS, dtype: int64

In [8]:
pd.set_option('display.max_rows', 500)
df2[['STATEFIP','COUNTYFIPS']].drop_duplicates()

,STATEFIP,COUNTYFIPS
0,1,97
4,1,0
9,1,15
17,1,81
26,1,73
30,1,3
57,1,55
115,1,117
47476,2,20
47480,2,0


In [9]:
df2.iloc[525657:525757]

,YEAR,DATANUM,SERIAL,SUBSAMP,HHWT,HHTYPE,REGION,STATEFIP,COUNTYFIPS,OWNERSHP,...,LABFORCE,OCC,OCC1990,WKSWORK2,INCTOT,FTOTINC,MIGRATE5,MIGRATE1,MIGPLAC5,MIGPLAC1
525657,2015,1,214637,95,156,1,41,8,69,1,...,2,220,22,6,50000,80000,NaN,1,NaN,0
525658,2015,1,214637,95,156,1,41,8,69,1,...,2,4510,458,6,30000,80000,NaN,1,NaN,0
525659,2015,1,214638,33,76,1,41,8,69,1,...,1,2850,183,0,33000,110000,NaN,1,NaN,0
525660,2015,1,214638,33,76,1,41,8,69,1,...,1,430,22,0,77000,110000,NaN,1,NaN,0
525661,2015,1,214639,38,22,0,41,8,0,0,...,2,9820,905,6,25400,9999999,NaN,1,NaN,0
525662,2015,1,214640,25,128,9,41,8,0,2,...,2,430,22,6,22000,22000,NaN,1,NaN,0
525663,2015,1,214640,25,128,9,41,8,0,2,...,2,5810,385,6,27800,22000,NaN,1,NaN,0
525664,2015,1,214641,42,73,1,41,8,0,1,...,1,0,999,0,14900,57050,NaN,1,NaN,0
525665,2015,1,214641,42,73,1,41,8,0,1,...,1,860,24,0,42150,57050,NaN,1,NaN,0
525666,2015,1,214642,80,44,1,41,8,0,1,...,2,430,22,6,75000,76500,NaN,1,NaN,0


In [13]:
pd.set_option('display.max_columns', None)
df

,YEAR,SERIAL,SUBSAMP,HHWT,HHTYPE,REGION,STATEFIP,COUNTYFIPS,OWNERSHP,RENT,VALUEH,PERNUM,PERWT,SEX,AGE,MARST,RACE,HISPAN,BPL,SCHOOL,EDUC,EDUCD,EMPSTAT,OCC,OCC1990,INCTOT,MIGRATE5,MIGRATE1,MIGPLAC5,MIGPLAC1,STATEFIP_orig,OCC1990_orig,EDUCD_orig,OCC_orig,BPL_orig
0,2015,1,26,48,Married-couple family household,East South Central Div.,Alabama,97,Rented,NaN,NaN,1,48,Male,56,"Married, spouse present",Black/Negro,Not Hispanic,Alabama,"No, not in school",Grade 11,Grade 11,Employed,NaN,Clergy and religious workers,3450.0,NaN,Same house,NaN,None,1,176,50,2040,1
1,2015,1,26,48,Married-couple family household,East South Central Div.,Alabama,97,Rented,NaN,NaN,2,53,Female,61,"Married, spouse present",Black/Negro,Not Hispanic,Alabama,"No, not in school",1 year of college,"1 or more years of college credit, no degree",Not in labor force,NaN,None,14000.0,NaN,Same house,NaN,None,1,999,71,0,1
2,2015,1,26,48,Married-couple family household,East South Central Div.,Alabama,97,Rented,NaN,NaN,3,81,Female,8,Never married/single,Black/Negro,Not Hispanic,Alabama,"Yes, in school",Nursery school to grade 4,Grade 2,None,NaN,None,NaN,NaN,Moved within state,NaN,Alabama,1,999,15,0,1
3,2015,1,26,48,Married-couple family household,East South Central Div.,Alabama,97,Rented,NaN,NaN,4,81,Male,61,Divorced,Black/Negro,Not Hispanic,New Jersey,"No, not in school",1 year of college,"1 or more years of college credit, no degree",Employed,NaN,"Truck, delivery, and tractor drivers",35000.0,NaN,Same house,NaN,None,1,804,71,9600,34
4,2015,2,76,65,"Female householder, living alone",East South Central Div.,Alabama,0,Rented,210.0,NaN,1,65,Female,52,Divorced,Two major races,Not Hispanic,Alabama,"No, not in school","Grade 5, 6, 7, or 8",Grade 8,Not in labor force,NaN,None,8800.0,NaN,Same house,NaN,None,1,999,26,0,1
5,2015,3,2,148,"Male householder, living alone",East South Central Div.,Alabama,0,Rented,NaN,NaN,1,148,Male,84,Widowed,White,Not Hispanic,Alabama,"No, not in school",4 years of college,Bachelor's degree,Not in labor force,NaN,None,13200.0,NaN,Same house,NaN,None,1,999,101,0,1
6,2015,4,92,36,Married-couple family household,East South Central Div.,Alabama,97,Owned or being bought (loan),NaN,275000.0,1,36,Male,53,"Married, spouse present",White,Not Hispanic,Alabama,"No, not in school",4 years of college,Bachelor's degree,Employed,NaN,Supervisors and proprietors of sales jobs,150000.0,NaN,Same house,NaN,None,1,243,101,4710,1
7,2015,4,92,36,Married-couple family household,East South Central Div.,Alabama,97,Owned or being bought (loan),NaN,275000.0,2,50,Female,53,"Married, spouse present",White,Not Hispanic,Alabama,"No, not in school",1 year of college,"1 or more years of college credit, no degree",Employed,NaN,"Teachers , n.e.c.",6000.0,NaN,Same house,NaN,None,1,159,71,2540,1
8,2015,4,92,36,Married-couple family household,East South Central Div.,Alabama,97,Owned or being bought (loan),NaN,275000.0,3,36,Male,16,Never married/single,White,Not Hispanic,Alabama,"Yes, in school",Grade 10,Grade 10,Not in labor force,NaN,None,0.0,NaN,Same house,NaN,None,1,999,40,0,1
9,2015,5,81,60,"Male householder, living alone",East South Central Div.,Alabama,15,Owned or being bought (loan),NaN,45000.0,1,61,Male,83,Divorced,White,Not Hispanic,Alabama,"No, not in school","Grade 5, 6, 7, or 8",Grade 7,Not in labor force,NaN,None,14000.0,NaN,Same house,NaN,None,1,999,25,0,1


In [12]:
options(repr.matrix.max.cols=200)

SyntaxError: keyword can't be an expression (<ipython-input-12-9364655ba9a6>, line 1)